# 작물 잎 사진으로 질병 분류하기

목표 : 전이학습 개념 확립 (CNN모델과 모델 학습 방식에서 어떤 차이점과 장점이 있는지)

데이터수:  40000

각 클래스 명 : 작물의 종류, 질병 종류 (healthy - 해당 작물이 건강함)

ex) potato -> potato early blight/ potato late bright / potato_healthy

두가지 모델을 구축후 성능을 비교평가.
1. CNN 구조를 활용하여 가장 기본적 베이스라인 모델 구축
2. 미리 학습되어있는 모델을 사용하는 transfer learning 기법을 호라용하여 모델을 학습시킨 후, 베이스라인 모델과 비교.

두 가지 모델의 성능뿐 아니라 구조와 활용방법 비교


데이터 셋 ( 학습 / 검증 / 테스트 )

# 데이터로드 및 분할폴더생성( 학습 / 검증 / 테스트 )

In [5]:
import os
import shutil
 
original_dataset_dir = './dataset'                 #원본데이터셋 위치한 경로 지정
classes_list = os.listdir(original_dataset_dir)    #os.listdir() 해당 경로 하위에 있는 모든 폴더 목록을 가져오는 메서드. 이 경우 폴더 목록은 클래스의 목록에 해당하므로 이것을 Class_list로 저장
 
base_dir = './splitted'                            #나눈 데이터를 저장할 폴더 생성
os.mkdir(base_dir)
 
train_dir = os.path.join(base_dir, 'train')        #분리 후 각 데이터를 저장할 하위폴더 train, val, test 생성
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'val')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

for cls in classes_list:     
    os.mkdir(os.path.join(train_dir, cls))
    os.mkdir(os.path.join(validation_dir, cls))
    os.mkdir(os.path.join(test_dir, cls))

# 분할폴더에 데이터 저장

In [6]:
import math
 
for cls in classes_list:                                                                          #모든 클래스에 대한 작업 반복
    path = os.path.join(original_dataset_dir, cls)
    fnames = os.listdir(path)                                                                     #path 위치에 존재하는 모든 이미지파일의 목록을 변수 fnames에 저장
 
    train_size = math.floor(len(fnames) * 0.6)
    validation_size = math.floor(len(fnames) * 0.2)
    test_size = math.floor(len(fnames) * 0.2)                                                     # 6:2:2 비율로 지정
    
    train_fnames = fnames[:train_size]                                                            #Train 데이터에 해당하는 파일의 이름을 train_fnames에 저장
    print("Train size(",cls,"): ", len(train_fnames))

    for fname in train_fnames:                                                                    
        src = os.path.join(path, fname)                                                           #복사할 원본파일 경로 지정
        dst = os.path.join(os.path.join(train_dir, cls), fname)                                   #복사한후 저장할 파일의 경로를 지정
        shutil.copyfile(src, dst)                                                                 #src경로에 해당하는 파일을 dst 경로에 저장
        
    validation_fnames = fnames[train_size:(validation_size + train_size)]
    print("Validation size(",cls,"): ", len(validation_fnames))
    for fname in validation_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(validation_dir, cls), fname)
        shutil.copyfile(src, dst)
        
    test_fnames = fnames[(train_size+validation_size):(validation_size + train_size +test_size)]

    print("Test size(",cls,"): ", len(test_fnames))
    for fname in test_fnames:
        src = os.path.join(path, fname)
        dst = os.path.join(os.path.join(test_dir, cls), fname)
        shutil.copyfile(src, dst)

Train size( Strawberry___healthy ):  273
Validation size( Strawberry___healthy ):  91
Test size( Strawberry___healthy ):  91
Train size( Grape___Black_rot ):  708
Validation size( Grape___Black_rot ):  236
Test size( Grape___Black_rot ):  236
Train size( Potato___Early_blight ):  600
Validation size( Potato___Early_blight ):  200
Test size( Potato___Early_blight ):  200
Train size( Cherry___Powdery_mildew ):  631
Validation size( Cherry___Powdery_mildew ):  210
Test size( Cherry___Powdery_mildew ):  210
Train size( Tomato___Target_Spot ):  842
Validation size( Tomato___Target_Spot ):  280
Test size( Tomato___Target_Spot ):  280
Train size( Peach___healthy ):  216
Validation size( Peach___healthy ):  72
Test size( Peach___healthy ):  72
Train size( Potato___Late_blight ):  600
Validation size( Potato___Late_blight ):  200
Test size( Potato___Late_blight ):  200
Train size( Tomato___Late_blight ):  1145
Validation size( Tomato___Late_blight ):  381
Test size( Tomato___Late_blight ):  381

# 베이스라인모델(CNN) 학습을 위한 준비

In [8]:
import torch
import os
 
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
BATCH_SIZE = 256 
EPOCH = 30 

import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder 
 
transform_base = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])  #transforms.compose()는 이미지 데이터의 전처리, augmentation등의 과정에서 주로사용
                                                                                          #이미지 증강에는 대표적으로 좌우반전, 밝기조절, 이미지 임의 확대 등이 있음.
                                                                                          #증강을 통해 이미지에 노이즈를 주어 더욱 강건한 모델 생성 가능
                                                                                          #Resize()-이미지크기를 64x64로 조정.
                                                                                          #ToTensor()-이미지를 텐서형태로 변환하고 모든값을 0~1사이로 정규화
train_dataset = ImageFolder(root='./splitted/train', transform=transform_base)            #ImageFolder()-하나의클래스(종류)가 하나의폴더에 대응하는 데이터셋형태 데이터셋 불러올때
                                                                                          #transform 옵션에는 데이터를 불러온 후 전처리 또는 증강할 방법을 지정.
val_dataset = ImageFolder(root='./splitted/val', transform=transform_base)                #검증을 위한 데이터를 val 폴더에 접근하여 불러오고, 학습데이터와 동일하게 전처리

In [9]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
#dataloader는 불러온 이미지 데이터를 주어진 조건에 따라 미니배치 단위로 분리함. 학습과정에 사용될 dataloader는 train_dataset을 이용하여 생성.

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
#검증과정에 사용될 dataloader는 val_dataset을 이용하여 생성.

# 베이스라인 모델 설계

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
 
class Net(nn.Module): 
  
    def __init__(self):                                               #모델에서 사용할 모든 layer 정의
    
        super(Net, self).__init__()                                   #nn.Module내에 있는 메서드 상속받아 사용

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)                   #(4) 첫번째 2D convolution layer 정의. 입력된 이미지데이터에서 2차원 conv연산을 하는 필터에 해당
                                                                      #(입력채널수, 출력채널수, 커널크기)
        self.pool = nn.MaxPool2d(2,2)                                 #(5)2차원 maxpooling을 실행하는 layer정의. (커널크기, stride)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)                  #(6)(입력채널수, 출력채널수, 커널크기)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)                  #(7)(입력채널수, 출력채널수, 커널크기)

        self.fc1 = nn.Linear(4096, 512)                               #flatten이후에 사용될 첫번째 fully connected layer정의 (입력채널수는 flatten의 출력 채널수와 동일)
        self.fc2 = nn.Linear(512, 33)                                 #(9)flatten이후에 사용될 두번째 fully connected layer정의 (마지막 레이어이므로 출력채널수는 분류클래스 수와 동일)
    
    def forward(self, x):                                             #모델이 학습데이터 입력받아 forward propagation을 실행시켜 output계산 과정 정의
    
        x = self.conv1(x)                                             #(11)4에서 정의한 conv1 layer를 이용해 convolution연산을 진행한 후 feature map을 생성
        x = F.relu(x)                                                 #11에서 conv 연산을 통해 생성된 피처맵값에 비선형 활성 함수인 ReLU를 적용
        x = self.pool(x)                                              #maxpooling 적용
        x = F.dropout(x, p=0.25, training=self.training)              #maxpooling 결과값에 dropout적용, p는 dropout 비율을 의미함.
                                                                      #p=0.25 이므로 25%의 노드를 dropout한다는 의미 training=self.trainiing - 학습모드와 검증모드일때 각각 달리 적용
                                                                      #학습과정에선 일부노드 랜덤하게 제외, 평가과정에선 모든 노드 사용해야하기 때문
        x = self.conv2(x)                                             #6에서 정의한 conv2 layer를 이용해 conv 연산을 진행 후 feature map 생성
        x = F.relu(x)                                                 #7에서 정의한 conv3 layer를 이용해 conv 연산을 진행 후 feature map 생성
        x = self.pool(x)                                              
        x = F.dropout(x, p=0.25, training=self.training)

        x = self.conv3(x)                                             #7에서 정의한 conv3 layer를 이용해 conv 연산을 진행 후 feature map 생성
        x = F.relu(x) 
        x = self.pool(x) 
        x = F.dropout(x, p=0.25, training=self.training)

        x = x.view(-1, 4096)                                          #(17)생성된 feature map을 1차원으로 펼치는 과정인 flatten 수행
        x = self.fc1(x)                                               #17에서 flatten된 1차원 텐서를 8에서 정의한 fc1에 통과시킴
        x = F.relu(x)                                                 
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)                                               #9에서 정의한 모델의 마지막 layer. 클래스 개수에 해당하는 33개의 출력값을 가짐

        return F.log_softmax(x, dim=1)                                #마지막 레이어의 33개 결과값에 softmax()함수를 적용하여 데이터가 각 클래스에 속할 확률을 output값으로 출력

model_base = Net().to(DEVICE)                                         #정의한 cnn모델 net()의 새로운 객체를 생성합니다. to(DEVICE)를 통해 모델을 현재 사용중인 장비에 할당
optimizer = optim.Adam(model_base.parameters(), lr=0.001)             #옵티마이저는 아담으로 설정하고, learning rate는 0.001로 설정

# 모델학습을 위한 함수

In [11]:
def train(model, train_loader, optimizer):
    model.train()                                                    #입력받는 모델을 학습 모드로 설정
    for batch_idx, (data, target) in enumerate(train_loader):        #앞서 정의했던 train_loader에는 (data, target)형태가 미니 배치 단위로 묶여 있음.
                                                                     #train_loader에 enumerate함수를 적용하여 batch_idx, (data, target)형태로 반복 가능한 객체 생성되어 for 실행
        data, target = data.to(DEVICE), target.to(DEVICE)            #data와 target변수를 사용중인 장비에 할당
        optimizer.zero_grad()                                        #이전 batch의 gradient값이 optimizer에 저장되어 있으므로 optimizer 초기화
        output = model(data)                                         #데이터를 모델에 입력하여 output값 계산
        loss = F.cross_entropy(output, target)                       #(6)모델에서 계산한 output값이 예측값과 target값 사이의 Loss를 계산. 분류문제에 적합한 cross entropy loss 사용
        loss.backward()                                              #(7)6에서 계산한 loss값 바탕으로 back propagation을 통해 계산한 gradient값을 각 파라미터에 할당
        optimizer.step()                                             #7에서 각 parameter에 할당된 gradient값을 이용해 모델의 parameter업데이트

# 모델 평가를 위한 함수

In [12]:
def evaluate(model, test_loader):                                               
    model.eval()                                                                #입력받는 모델을 평가 모드로 설정
    test_loss = 0                                                               #미니배치별로 loss를 합산해서 저장할 변수인 test_loss를 선언하고 0으로 초기화
    correct = 0                                                                 #올바르게 예측한 데이터의 수를 세는 변수인 correct를 선언하고 0으로 초기화
    
    with torch.no_grad():                                                       #모델을 평가하는 단계에서는 모델의 parameter를 업데이트 하지 않아야하므로 with torch.no_grad()로 파라미터 업데이트 중단
        for data, target in test_loader:                                        #앞에서 학습한 것과 같이 train_loaderdpsms (data, target)형태가 미니 배치 단위로 묶임. for를 통해 데이터와 대응하는 label값에 접근
            data, target = data.to(DEVICE), target.to(DEVICE)                   #data와 target 변수를 사용중인 장비로 할당
            output = model(data)                                                #데이터를 모델에 입력하여 output값 계산
            
            test_loss += F.cross_entropy(output,target, reduction='sum').item() #모델에서 계산한 output값인 예측값과 target값 사이의 loss계산. 성능평가 과정에서도 cross entropy loss 함수를 사용
 
            
            pred = output.max(1, keepdim=True)[1]                               #모델에 입력된 Test 데이터가 33개의 클래스에 속할 각각의 확률값이 output으로 출력됨. 이중 가장 높은 값을 가진 인덱스를 예측값으로 저장
            correct += pred.eq(target.view_as(pred)).sum().item()               #target.view_as(pred)를 통해 target Tensor 구조를 pred Tensor와 같은 모양으로 정렬.
                                                                                #view_as()메서드는 적용 대상 텐서를 메서트에 입력되는 텐서 모양대로 재정렬하는 함수.
                                                                                #view()함수는 정렬하고 싶은 텐서의 모양을 숫자로 직접 지정해야하는 것에서 차이 발생.
                                                                                #eq()메서드는 객체간 비교 연산자로, pred.eq(target.view_as(pred))는 pred와 target.view_as(pred)의 값이 일치하면 1, 아니면 0
                    
   
    test_loss /= len(test_loader.dataset)                                       #모든 미니 배치에서 합한 loss값을 batch수로 나누어 미니배치마다 계산된 정확도값의 평균을 구함
    test_accuracy = 100. * correct / len(test_loader.dataset)                   #모든 미니 배치에서 합한 정확도 값을 batch수로 나누어 미니배치마다 계산된 정확도 값의 평균을 구함
    return test_loss, test_accuracy                                             #측정한 testloss와 정확도 반환

# 모델 학습 실행하기

In [13]:
import time
import copy
 
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0                                                                           #정확도가 가장 높은 모델의 정확도를 저장하는 변수 best_acc를 선언후 값을 0으로 초기화
    best_model_wts = copy.deepcopy(model.state_dict())                                       #정확도 가장 높은 모델을 저장할 변수 best_model_wts 선언
 
    for epoch in range(1, num_epochs + 1): 
        since = time.time()                                                                  #한 epoch당 소요 시간 측정위해 해당 epoch시작할 때 시각 저장
        train(model, train_loader, optimizer)                                                #앞서 정의한 train()함수를 이용하여 모델 학습
        train_loss, train_acc = evaluate(model, train_loader)                                #앞서 정의한 evaluate()함수를 이용하여 해당 epoch에서의 학습 loss와 정확도를 계산
        val_loss, val_acc = evaluate(model, val_loader)                                      #앞서 정의한 evaluate()함수를 이용하여 해당 epoch에서의 검증 loss와 정확도를 계산
        
        if val_acc > best_acc:                                                               #현재 epoch의 검증 정확도가 최고 정확도보다 높다면 best_acc를 현재 Epoch의 검증 정확도로 업데이트
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())                               #해당 모델을 best_model_wts에 저장
        
        time_elapsed = time.time() - since                                                   #한 epoch당 소요된 시간 계산 
        print('-------------- epoch {} ----------------'.format(epoch))                      
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))         #해당 Epoch의 학습 loss와 정확도를 출력
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))               #해당 Epoch의 검증 loss와 정확도를 출력
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))  #한 epoch당 소요된 시간을 출력
    model.load_state_dict(best_model_wts)                                                    #최종적으로 정확도가 가장 높은 모델을 불러오고
    return model                                                                             #불러온 모델 반환
 

base = train_baseline(model_base, train_loader, val_loader, optimizer, EPOCH)  	             #앞서 정의한 train_baseline()함수를 이용해 baseline모델 학습 
torch.save(base,'baseline.pt')                                                               #학습 완료된 모델 저장

-------------- epoch 1 ----------------
train Loss: 1.7331, Accuracy: 48.42%
val Loss: 1.7330, Accuracy: 48.32%
Completed in 2m 23s
-------------- epoch 2 ----------------
train Loss: 1.0845, Accuracy: 67.31%
val Loss: 1.1103, Accuracy: 66.85%
Completed in 2m 23s
-------------- epoch 3 ----------------
train Loss: 0.8228, Accuracy: 74.05%
val Loss: 0.8616, Accuracy: 72.52%
Completed in 2m 23s
-------------- epoch 4 ----------------
train Loss: 0.5779, Accuracy: 82.52%
val Loss: 0.6228, Accuracy: 81.14%
Completed in 2m 24s
-------------- epoch 5 ----------------
train Loss: 0.4897, Accuracy: 84.93%
val Loss: 0.5412, Accuracy: 83.39%
Completed in 2m 24s
-------------- epoch 6 ----------------
train Loss: 0.4264, Accuracy: 87.39%
val Loss: 0.4923, Accuracy: 84.52%
Completed in 2m 24s
-------------- epoch 7 ----------------
train Loss: 0.3790, Accuracy: 88.57%
val Loss: 0.4535, Accuracy: 86.39%
Completed in 2m 24s


KeyboardInterrupt: 

# 전이학습을 위한 데이터 준비 transfer learning

높은성능의 이미지 분류 모델 구축을 위해 많은 양의 질 좋은 데이터 필요.

but 대개 양질의 데이터 대량 구하기 힘듦.

-> 대량 데이터셋으로 미리 학습된 모델 재활용 후 일부 조정하여 다른 주제의 이미지 분류 모델에 사용하면 효과적

대량의 데이터셋으로 미리 학습된 모델 : pre trained model

pre trained model을 조정하는 과정 : fine tuning

위 기법을 통틀어서 'transfer learning'.

------------------------------------------------

일반적으로 pytorch는 torchvision.models 패키지에서 imagenet데이터를 학습해놓은

alexnet, vgg, resnet, squeezenet, densenet, inception v3, googlenet,

shufflenetv2, mobilenet v2, resnext, wide resnet, mnasnet 등 모델을 바로 불로 올 수 있도록 지원. 

torchvision.models 공식 문서 참조

------------------------------

베이스라인 모델의 경우 학습 진행시 초기 parameter값은 랜덤.

Transfer learning의 경우 미리 학습된 모델의 params 불러오고 학습 과정에서 업데이트

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize([64,64]),                    #transforms.compose()는 이미지 데이터 전처리, 증강 등 과정에서 사용되는 메서드, 이미지크기 64x64로 조정
        transforms.RandomHorizontalFlip(),             #이미지 증강에 해당 무작위로 좌우 반전 (괄호)안에 param p를 입력하여 반전되는 이미지의 비율 설정가능
        ransforms.RandomVerticalFlip(),                #기본 디폴트값 0.5  #무작위 상하반전
        transforms.RandomCrop(52),                     #이미지 일부 무작위로 잘라내어 52x52사이즈로 변경/ centercrop, fivecrop등 다양한 방법 존재
        transforms.ToTensor(),                         #이미지 탠서 형태로 변환하고 모든 숫자를 0에서 1사이로 변경.
        transforms.Normalize([0.485, 0.456, 0.406],    #이미지가 탠서 전환된 이후 정규화 시행. 정규화 위해서는 평균과 표준편차 값 필요.
                             [0.229, 0.224, 0.225]) ]),#normalize()메서드 내의 첫번째 대괄호 []는 각각 r, g, b채널 값에서 정규화 적용할 평균 값 의미
                                                       #두번째 [] 표준편차 값 의미. 사용된 평균값과 표준편차값은 pre trained model의 학습에 사용된 ImageNet 데이터 값임.
                                                       #입력 데이터 정규화는 모델을 최적화하고 local minimum에 빠지는 것을 방지하는데 도움이 됩니다.
    
    'val': transforms.Compose([transforms.Resize([64,64]),  # 검증에 사용되는 데이터에는 학습에 적용했던 전처리에서 증각에 해당하는 부분 제외한 나머지부분 동일 적용
        transforms.RandomCrop(52), transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) ])
}

data_dir = './splitted'   #학습데이터와 검증데이터를 불러올 폴더 경로 설정
image_datasets = {x: ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in ['train', 'val']}
#ImageFolder()메서드는 데이터셋 불러오는 메서드. root 옵션에 데이터를 불러올 경로 설정. transform 옵션에는 데이터 불러온 후 전처리 또는 증강 방법 지정.
#앞서 정의한 transfrom_base로 옵션 지정. 훈련 검증과정에서 각각의 과정에 맞는 데이터를 편리하게 불러오고자 딕셔너리 형태로 구성.

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE, shuffle=True, num_workers=4) for x in ['train', 'val']} 
#Dataloader는 불러온 이미지 데이터를 주어진 조건에 따라 미니 배치 단위로 분리하는 역할 수행함. image_datasets를 이용하여 생성하며,
#셔플을 해야 모델 학습시 label정보 순서 기억하는 것 방지. 훈련 검증과정에서 각각 단계맞는 데이터 편리하게 불러오기 위해 딕셔너리로 구성

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
#활용을 위해 학습데이터와 검증 데이터의 총 개수 각각저장. 훈련 검증과정에서 각각 단계에 맞는 데이터 편리하게 불러오고자 딕셔너리로 저장

class_names = image_datasets['train'].classes #이후 활용을 위해 33개의 클래스 이름 목록을 저장.

# pre-Trained model 불러오기

In [ ]:
from torchvision import models
 
resnet = models.resnet50(pretrained=True)  #true로 설정시 미리 학습된 모델의 params값을 그대로 가져옴. false -> 구조만 가져오고 param랜덤
num_ftrs = resnet.fc.in_features           #데이터 33개 클래스로 분류하므로 모델 마지막 레이어 출력 채널 수 33개여야함. 하지만 resnet50은 출력채널수 33개아님.
                                           #따라서 마지막 fully connected layer 대신 33개의 새로운 layer추가 필요하여 마지막 layer의 입력 채널 수 저장
                                           #in_features는 해당 layer의 입력 채널 수를 의미.
resnet.fc = nn.Linear(num_ftrs, 33)        #불러온 모델의 마지막 fully connected layer를 새로운 layer로 교체. 입력 채널수는 기존 layer와 동일하고 출력은 33으로 설정
resnet = resnet.to(DEVICE)                 #모델을 현재 사용중인 장비에 할당
 
criterion = nn.CrossEntropyLoss()          #모델 학습시 사용되는 loss함수 지정 변수. 베이스라인 모델과 동일하게 cross entropy loss 사용
optimizer_ft = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=0.001)
# 앞서 학습한 베이스라인 모델에서는 모든 파라미터 업데이트 했지만, 이 모델에선 설정한 일부 레이어의 파라미터만을 업데이트해야함.
# ->따라서 filter()와 lambda표현식을 사용하여 requires_grad = True로 설정된 layer의 params에만 적용.
 
from torch.optim import lr_scheduler 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
#StepLR메서드는 epoch에 따라 learning rate를 변경하는 역할. 7 epoch마다 0.1씩 곱해 learning rate를 감소시킨다는 의미.


# pre-Trained model의 일부 layer freeze하기

pre trained model을 fine tuning 하는 과정에서 일부 layer는 학습 시 업데이트 안되게 하고 일부 layer는 업데이트 되도록 설정.

일부 layer 업데이트 안되게 고정하는 것 : 'layer를 freeze한다'

이미지 분류모델은 먼저 Lowlevel Feature 학습(모서리 같이 작은 지역패턴 학습) 후 -> Highlevel feature 학습(low level feature로 구성된 더 큰 패턴)

다른 종류의 이미지라도 낮은 수준의 특징은 상대적으로 서로 비슷할 가능성이 큼.

-> fine tuning 과정에서 마지막 layer인 분류기와 가까운 layer부터 원하는만큼 학습 과정에서 업데이트함.

이때 freeze하는 layer 수는 데이터셋 크기와 pretrained model에 사용된 데이터셋과의 유사성을 고려하여 결정.

4가지 case

(자신이 가진 데이터셋 크기와 데이터셋이 pre-trainedmodel의 학습에 사용된 데이터와 얼마나 유사한지에 따라 4가지로 구분

모델 구조는 크게 이미지의 feature를 학습하는 convolutional base 부분과 이미지를 분류하는 classifier부분으로 나뉨.

분류하는 이미지가 다를경우 classifier 부분을 변경해야함.

1. - 자신이 가진 데이터 수도 많고, 원래 학습에 이용된 데이터와의 유사도가 높을 경우. -> 일부 레이어만 freeze

=> 데이터의 유사도가 높기 때문에 상대적으로 적은 수의 layer만 업데이트 해도 높은 성능 낼 수 있음.

2. - 가진 데이터 수는 많지만, 원래 학습에 이용된 데이터와의 유사도 낮음 -> 전체 레이어 unfreeze

=> 모든 레이어가 학습과정에서 업데이트 되지만, 모델의 효과적인 구조는 그대로 유지되고 불러온 파라미터값에서 조금씩 업데이트하며 조정하는 점이 처음부터 모델 구성하는 방법과 차이임.

3. - 자신이 가진 데이터 적고, 유사도도 낮음 -> 일부 레이어 freeze

=> 2의 경우 모델의 크기에 비해 데이터 수가 충분하여 전체 layer를 재학습해도 과적합 위험이 상대적으로 낮음. 3사분면의 경우 데이터 충분하지 않으므로 일부 layer만 업데이트 하는게 효과적.

4. - 데이터 수는 적지만, 유사도는 높음. -> convolutional base부분 전체를 freeze하고 classifier부분만 변경

=> 데이터 유사도가 높으므로 pre trained model이 학습한 많은 feature들을 그대로 사용할 수 있지만, 일부 layer를 재학습하기엔 과적합 위험이 상대적으로 크기 때문에 conv base 전체 freeze


In [ ]:
ct = 0                                    #해당 레이어가 몇번째 레이어이진 나타내는 변수 ct 선언
for child in resnet.children():           #children() - 자식 모듈을 반복 가능한 객체로 반환하는 메서드, resnet.children()- 생성한 resnet 모델의 모든 layer정보를 담고있음.
    ct += 1                               #다음 레이어를 지칭하도록 for문 반복마다 1씩 증가
    if ct < 6:                            #미리 학습된 모델의 일부 레이어만 업데이트하도록 파라미터 업데이트를 진행하지 않을 상위 레이어의 requires_grad = false로 지정
        for param in child.parameters():  #child.parameter는 각 layer의 parameter tensor를 의미. 각 Tensor에는 requires_grad옵션이 있고 기본값 True
            param.requires_grad = False   #layer의 번호가 6보다 작을 땐, parameter 업데이트 X

# 전이학습 모델 학습과 검증을 위한 함수

In [ ]:
def train_resnet(model, criterion, optimizer, scheduler, num_epochs=25):

    best_model_wts = copy.deepcopy(model.state_dict())                         #정확도가 가장 높은 모델 저장할 변수
    best_acc = 0.0                                                             #정확도 가장 높은 모델의 정확도 저장할 변수
    
    for epoch in range(num_epochs):
        print('-------------- epoch {} ----------------'.format(epoch+1))      #현재 진행중인 epoch 출력
        since = time.time()                                                    #한 epoch당 소요 시간 측정 위해 epoch 시작 시간 저장

        for phase in ['train', 'val']:                                         #한 epoch는 각각 학습과 검증단계 지님. for통해 한 epoch마다 학습모드와 검증 각각 실행 
            if phase == 'train':                                               #상황에 적합하게 모델을 학습모드로 설정
                model.train() 
            else:                                                              #상황에 적합하게 모델을 검증모드로 설정
                model.eval()     
 
            running_loss = 0.0                                                 #모든 데이터의 loss를 합산하여 저장할 변수인 running_loss 선언
            running_corrects = 0                                               #올바르게 예측한 경우의수를 세는 변수 선언
 
            
            for inputs, labels in dataloaders[phase]:                          #모델의 현재 모드에 해당하는 dataloader에서 데이터 입력받음
                inputs = inputs.to(DEVICE)                                     #데이터와 label을 현재 사용중인 장비에 할당
                labels = labels.to(DEVICE)  
                
                optimizer.zero_grad()                                          
                
                with torch.set_grad_enabled(phase == 'train'):                 #학습단계에서만 모델의 gradient를 업데이트하고 검증단계에서는 업데이트 하지 않아야함. set_grad_enabled이용
                    outputs = model(inputs)                                    #데이터를 모델에 입력하여 output값 계산
                    _, preds = torch.max(outputs, 1)                           #(15)모델에 입력된 test데이터가 33개 클래스에 속할 각각 확률값이 output으로 출력됌. 가장 높은 값 지닌 인덱스 예측값으로 저장
                    loss = criterion(outputs, labels)                          #(16)모델에서 계산한 output값인 예측값과 target값 사이의 Loss를 계산. 입력받는 Loss함수 이용하여 Loss 계산.
    
                    if phase == 'train':                                       #모델이 현재 학습 모드인 경우, 16에서 계산한 loss 바탕으로 back propagation을 통해 계산한 gradient값을 각 params에 할당후, 모델 params 업데이트
                        loss.backward()
                        optimizer.step()
 
                running_loss += loss.item() * inputs.size(0)                   #모든 데이터의 loss합산해서 저장하기 위해 하나의 미니 배치에 대해 계산된 loss값에 데이터수를 곱해 합산함. 이때, inputs.size(0)은 dataloader에서 전달되는 미니배치의 데이터 수를 의미하는 것으로 배치사이즈임. 
                running_corrects += torch.sum(preds == labels.data)            #15에서 모델을 통해 예측한 값과 target이 같으면 running_corrects를 1만큼 증가시키고, 같지 않으면 증가시키지 않음.
            if phase == 'train':                                               #한 epoch당 1번 모델이 현재 학습 단계일 경우에만 실행
                scheduler.step()
                l_r = [x['lr'] for x in optimizer_ft.param_groups]             #스케쥴러에 의해 learning rate가 조정되는 것을 직접 확인하는 부분
                                                                               #optimizer_ft.param_groups의 원소는 학습 과정에서의 parameter를 저장하고 있는 딕셔너리. 이중 learning rate에 해당하는 키인 'lr'이용하여 각 epoch의 learning rate를 불러옴.
                print('learning rate: ', l_r)
                       
            epoch_loss = running_loss/dataset_sizes[phase]                     #epoch의 loss를 계산하기 위해 running_loss를 미리 계산해둔 dataset_size로 나눔.                     
            epoch_acc = running_corrects.double()/dataset_sizes[phase]         #해당 epoch의 정확도를 계산하기 위해 running_corrects를 미리 계산해둔 dataset_size로 나눔.
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc)) #해당 epoch와 현재 모델의 단계, loss값, 정확도 출력
    
         
            if phase == 'val' and epoch_acc > best_acc:                        #검증단계에서 현재 epoch의 정확도가 최고 정확도보다 높다면, best_acc를 현재 epoch의 정확도로 업데이트후
                                                                               
                best_acc = epoch_acc                                           
                best_model_wts = copy.deepcopy(model.state_dict())             #해당 epoch모델을 best_model_wts에 저장
 
        time_elapsed = time.time() - since                                     #한 epoch당 소요된 시간 계산. 
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))                              
 
    model.load_state_dict(best_model_wts)                                      #정확도가 가장 높은 모델을 불러온 후 반환

    return model    
#학습을 위한 함수 구성 완료.

# 모델 학습 실행하기

In [ ]:
model_resnet50 = train_resnet(resnet, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=EPOCH) 
#앞서 정의한 train_resnet()함수를 이용해 Resnet50모델을 fine-tuning함
torch.save(model_resnet50, 'resnet50.pt')
#학습이 완룐된 모델인 model_resnet50을 'resnet50.pt'라는 이름의 파일로 저장

# 베이스라인 모델 평가를 위한 전처리

In [ ]:
transform_base = transforms.Compose([transforms.Resize([64,64]),transforms.ToTensor()])
test_base = ImageFolder(root='./splitted/test',transform=transform_base)  
test_loader_base = torch.utils.data.DataLoader(test_base, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#베이스라인 모델 성능 평가를 위해 사용할 테스트데이터의 dataloader 생성.
#모델을 학습시킬 때 사용한 학습 및 검증 데이터와 동일한 방법으로 전처리 수행하고 배치사이즈도 동일하게 설정

# 전이학습 모델 평가를 위한 전처리

In [ ]:
transform_resNet = transforms.Compose([
        transforms.Resize([64,64]),  
        transforms.RandomCrop(52),  
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) 
    ])
    
test_resNet = ImageFolder(root='./splitted/test', transform=transform_resNet) 
test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#전이학슴 모델 성능평가를 위해 사용할 테스트데이터의 dataloader를 생성.
#마찬가지로 모델을 학습시킬때 사용한 학습 및 검증 데이터와 동일방법으로 전처리 수행하고 배치사이즈도 동일하게 설정

# 베이스라인 모델 성능 평가하기

In [ ]:
baseline=torch.load('baseline.pt')  #저장했던 베이스라인 모델 불러옴
baseline.eval()   #모델을 평가모드로 설정
test_loss, test_accuracy = evaluate(baseline, test_loader_base) #evaluate함수 이용하여 테스트데이터에 대한 정확도 측정

print('baseline test acc:  ', test_accuracy) #평가 정확도 출력

# 전이학습 모델 성능 평가하기

In [ ]:
resnet50=torch.load('resnet50.pt')  #저장된 전이학습 모델 불러옴
resnet50.eval()  #모델을 평가 모드로 설정
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet) #앞서 정의한 evaluate함수 이용하여 테스트 데이터에 대한 정확도 측정

print('ResNet test acc:  ', test_accuracy) #평가 정확도 출력

결과

베이스라인 모델보다 파인튜닝한 resnet50모델이 더 높은 정확도 보유. 93% vs 97%

=> 모델을 직접 구축하여 처음부터 학습시키는 것보다 많은 양의 데이터셋으로 미리 학습된 모델을 불러와 일부를 fine tuning하는 것이 더 높은 예측 성능을 냄.

why?

1. 데이터 수와 질.

베이스라인 모델 : 40,000 데이터셋 나쁘지 않은 좋은 성능

finetuning resnet50 : 약14,000,000 데이터셋 다양한 이미지의 feature가 학습되어있음.(강아지 고양이, 사자, 등)

2. 파라미터 시작점

베이스라인 모델 : 초기 파라미터 랜덤

finetuning resnet50 : 수백만장 이미지 통해 미리 학습한 모델의 파라미터 이용

결론 : 전이학습은 이미지, 영상, 자연어 등 여러 분야에 다양하게 사용됌.
단순 분류외 물체의 존재 부분 파악하는 localization작업 or 하나의 이미지에 여러 물체 탐지하는 obj detection, 물체 경계를 정확히 파악하는 segmentation기술 등에도 사용.